# Twitter Sentiment Analysis

The dataset used is "Sentiment140" which can be accessed at the following link"
> https://www.kaggle.com/kazanova/sentiment140

The architecture used for sentiment analysis is "Word Embeddings" whose guide can be viewed at the following link:
>https://www.tensorflow.org/text/guide/word_embeddings

## Creating classes



In [1]:
class Sentiment:
    Negative = "Negative"
    Neutral = "Neutral"
    Positive = "Positive"


class Tweet:
    def __init__(self, tweet, score):
        self.Tweet = tweet
        self.Score = score
        self.Sentiment = get_sentiment(score)
        
    def get_sentiment(score):
        if score == 0 or score == 1:
            return Sentiment.Negative
        elif score == 2 or score == 3:
            return Sentiment.Neutral
        else:
            return Sentiment.Positive
    
class TweetMethods:
    def __init__(self, Tweet):
        self.tweet = tweet
    
    def get_tweet(self):
        return [x.tweet for x in self.tweet]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.tweet]
        